In [26]:
list_egos = [line[:-1] for line in open('../Data/egos_positions_missing', 'r')]

In [27]:
from igraph import *
import csv

class Enumerate(object):
    DICT_PATTERNS = {
        '[1, 1]' : 1,
        '[1, 1, 2]' : 2,
        '[2, 2, 2]' : 3,
        '[1, 1, 2, 2]' : 4,
        '[1, 1, 1, 3]' : 5,
        '[1, 2, 2, 3]' : 6,
        '[2, 2, 2, 2]' : 7,
        '[2, 2, 3, 3]' : 8,
        '[3, 3, 3, 3]' : 9,
        '[1, 1, 2, 2, 2]' : 10,
        '[1, 1, 1, 1, 4]' : 11,
        '[1, 1, 1, 2, 3]' : 12,
        '[1, 2, 2, 2, 3]' : (1,2,(13,17)),
        '[1, 1, 2, 3, 3]' : 14,
        '[1, 1, 2, 2, 4]' : 15,
        '[2, 2, 2, 2, 2]' : 16,
        '[1, 2, 3, 3, 3]' : 18,
        '[1, 2, 2, 3, 4]' : 19,
        '[2, 2, 2, 2, 4]' : 20,
        '[2, 2, 2, 3, 3]' : (3,3,(21,22)),
        '[1, 3, 3, 3, 4]' : 23,
        '[2, 2, 3, 3, 4]' : 24,
        '[2, 2, 2, 4, 4]' : 25,
        '[2, 3, 3, 3, 3]' : 26,
        '[3, 3, 3, 3, 4]' : 27,
        '[2, 3, 3, 4, 4]' : 28,
        '[3, 3, 4, 4, 4]' : 29,
        '[4, 4, 4, 4, 4]' : 30
    }

    DICT_POSITIONS = [
        {1 : 1}, #1
        {1 : 2, 2 : 3}, #2
        {2 : 4}, #3
        {1 : 5, 2: 6}, #4
        {1 : 7, 3 : 8}, #5
        {1 : 9, 2 : 10, 3 : 11}, #6
        {2 : 12},#7
        {2 : 13, 3 : 14},#8
        {3 : 15},#9
        {1 : 16, 2 : (1,(17,18))},#10
        {1 : 19, 4 : 20},#11
        {1 : (2,(21,22)), 2 : 23, 3 : 24},#12
        {1 : 25, 2 : (1,(26,27)), 3 : 28},#13
        {1 : 29, 2 : 30, 3 : 31},#14
        {1 : 32, 2 : 33, 4 : 34},#15
        {2 : 35},#16
        {1 : 36, 2 : (3,(38,37)), 3 : 39},#17
        {1 : 40, 2 : 41, 3 : (1,(42,43))},#18
        {1 : 44, 2 : 45, 3 : 46, 4 : 47},#19
        {2 : 48, 4 : 49},#20
        {2 : (2,(50, 51)), 3 : 52},#21
        {2 : 53, 3 : 54},#22
        {1 : 55, 3 : 56, 4 : 57},#23
        {2 : 58, 3 : 59, 4 : 60},#24
        {2 : 61, 4 : 62},#25
        {2 : 63, 3 : (2,(64,65))},#26
        {3 : 66, 4 : 67},#27
        {2 : 68, 3 : 69, 4 : 70},#28
        {3 : 71, 4 : 72},#29
        {4 : 73}#30
    ]

    def __init__(self, graph, k):
        self.patterns_tab = []
        self.positions_tab = []
        self._graph = graph
        self._k = k

    def create_list_neighbors(self):
        for v in self._graph.vs:
            v['list_neighbors'] = []
        for e in self._graph.es:
            if not e.source in self._graph.vs[e.target]['list_neighbors']:
                self._graph.vs[e.target]['list_neighbors'].append(e.source)
            if not e.target in self._graph.vs[e.source]['list_neighbors']:
                self._graph.vs[e.source]['list_neighbors'].append(e.target)
        for v in self._graph.vs:
            v['list_neighbors'].sort(reverse = True)

    def degree_distribution(self, graph_sub):
        result = []
        for v in graph_sub.vs:
            result.append(v.degree())
            v['d'] = result[v.index]
        result.sort()
        return result

    def disambiguate_pattern(self, graph_sub, new_pattern):
        for v in graph_sub.vs:
            if v['d'] == new_pattern[0]:
                for n in v.neighbors():
                    if n['d'] == new_pattern[1]:
                        return new_pattern[2][0]
        return new_pattern[2][1]

    def disambiguate_position(self, graph_sub, v, new_position):
        for n in v.neighbors():
            if n['d'] == new_position[0]:
                return new_position[1][0]
        return new_position[1][1]

    def index_pattern(self, graph_sub):
        new_pattern = self.DICT_PATTERNS[str(self.degree_distribution(graph_sub))]
        if type(new_pattern) != int :
            new_pattern = self.disambiguate_pattern(graph_sub, new_pattern)
        self.patterns_tab[new_pattern - 1] += 1
        new_positions = self.DICT_POSITIONS[new_pattern - 1]
        if new_pattern in [10, 12, 13, 17, 18, 21, 26] :
            for v in graph_sub.vs:
                new_position = new_positions[v['d']]
                if type(new_position) != int:
                    new_position = self.disambiguate_position(graph_sub, v, new_positions[v['d']])
                self.positions_tab[v['id_principal']][new_position - 1] += 1
        else:
            for v in graph_sub.vs:
                self.positions_tab[v['id_principal']][new_positions[v['d']] - 1] += 1

    def in_neighborhood_vsub(self, list_neighbors, length_vsub):
        for n in list_neighbors:
            if self._graph.vs[n]['id_sub'] != -1 and self._graph.vs[n]['id_sub'] != length_vsub-1:
                return True
        return False

    def add_vertex(self, graph_sub, vertex):
        vertex['id_sub'] = len(graph_sub.vs)
        graph_sub.add_vertex(name = vertex['name'], **{'id_principal' : vertex.index})

    def extend_subgraph(self, graph_sub, v, vext):
        if len(graph_sub.es) > 0 :
            self.index_pattern(graph_sub)
        if len(graph_sub.vs) == self._k:
            return
        while vext:
            w = vext.pop()
            vext2 = list(vext)
            self.add_vertex(graph_sub, w)
            for nei in w['list_neighbors']:
                u = self._graph.vs[nei]
                if u.index >= v.index:
                    if u['id_sub'] == -1 :
                        if not self.in_neighborhood_vsub(u['list_neighbors'], len(graph_sub.vs)):
                            vext2.append(u)
                    else:
                        graph_sub.add_edge(len(graph_sub.vs) - 1, u['id_sub'])
                else:
                    break

            self.extend_subgraph(graph_sub, v, vext2)
            graph_sub.delete_vertices(w['id_sub'])
            w['id_sub'] = -1

    def characterize_with_patterns(self):
        self.create_list_neighbors()
        self.patterns_tab = 30*[0]
        for v in self._graph.vs:
            self.positions_tab.append(73*[0])
            v['id_sub'] = -1
        for v in self._graph.vs:

            graph_sub = Graph.Formula()
            v['id_sub'] = 0
            if not 'name' in v.attributes():
                v['name'] = str(v.index)
            graph_sub.add_vertex(name = v['name'], **{'id_principal' : v.index, 'evol_class' : 1, 'pattern_sub' : 0})

            vext = []
            for nei in v['list_neighbors']:
                if nei > v.index:
                    vext.append(self._graph.vs[nei])

            if len(vext) > 0:
                self.extend_subgraph(graph_sub, v, vext)
            v['id_sub'] = -1
        return (self.patterns_tab, self.positions_tab)

In [28]:
def import_graph(ego):
    for folder in ['csa','p5','all','.']:
        if not os.path.isfile('../../Graphs/%s/%s.gml' % (folder, ego)):
            continue
        return Graph.Read_GML(os.path.expanduser('../../Graphs/%s/%s.gml' % (folder, ego)))

In [30]:
for ego in list_egos:
    try:
        g = import_graph(ego)
    except:
        continue
    print ego
    pt, ps = Enumerate(g, 5).characterize_with_patterns()
    with open('../Data/positions_per_alters/%s.csv' % ego, 'w') as to_write:
        csv_w = csv.writer(to_write, delimiter = ';')
        for alter in ps:
            csv_w.writerow(alter)
    print 'done'

8863489cda86677af796a9f7eeefc53b
done
887665a18a1204a0efd20d2a8a391180
done
888443cdefc2f0fd4cd38a506fc4a4db
done
889930e7d745e7b4e1957980ab055b19
done
88cb20cd47d4ae32603b9a536e098f80
done
89002229a65cbc934a62e38bc48bb2c9
done
89104d48ff0a0bf327c28064dc17f992
done
89258b6b93cb47d80e8ae7d2e527c454
done
893575eef49f6693a1196a7327d04672
done
89516675ef91027b869360cc9edc99d6
done
895dadf04d58a855e71f4c17099bba18
done
897d33865eca2c73a61b76f818e94069
done
89b247c9190862749dba1bcf29d70304
done
89b3490c5e8c6d0a75ddcbb469ebae41
done
89bc633635c71bc7dbd8c7be74cadefc
done
89e978133319678ba3bc788e767cba38
done
89fa7707578d1307b0e0e0842c97ad5c
done
8a0e9695c5143ef93cb947e1ab9e213b
done
8a1be069832652360f19a373580975a6
done
8a3c9103c66a0866d6394f1d0f0fe72b
done
8a56e708dd5f5f13f8e7f3383316cf3b
done
8a707a152dd95148378b990c476f73ab
done
8a78937eddb296996d770fa881a89954
done
8a83f086f779a78bbbd6aab2e7283b20
done
8a90aae1e81c58ca8bd372885efaa35c
done
8a97335101205a4cb5225afd82f44127
done
8a993c9451bf